In [4]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import time

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_˜

In [7]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}
#To iterate over comm_ids
id=int(0)

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()
comm_ids

['C00715532',
 'C00722108',
 'C00713958',
 'C00710087',
 'C00314575',
 'C00722454',
 'C00709899',
 'C00653816',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00433391',
 'C00723411',
 'C00709279',
 'C00712323',
 'C00694364']

In [74]:
## INDIVIDUAL DONATIONS TO MAINE SENATE CAMPAIGNS ##
#Initialize dataframe collector for itemized contribs
cycle='2020'
dfs=[]
commid=0

#Initialize query dictionary
querydict = {'per_page':'100'
              ,'sort':'contribution_receipt_date'
              ,'api_key':config.api_key
              ,'is_individual':'true'
              ,'two_year_transaction_period':cycle
              ,'last_index':[]
              ,'last_contribution_receipt_date':[]
              ,'committee_id':comm_ids[commid]
             }

#Page through results for each committee id
for x in comm_ids:
    
    #Get first payload for a candidate
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()
    
    #Print querydict to validate
    print(querydict)
    
    #Last page variables
    while r['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        df = json_normalize(r['results'])
        dfs.append(df)
        
        #For validation only
        print('page: '+str(commid)+','+str(r['pagination']['per_page'])+' records')
        
        #Assign last_index and date values, update querydict
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        #Update dictionary with new indices
        querydict.update([('last_index',last_index)
                        ,('last_contribution_receipt_date',last_date)])

        #Get next payload with updated dict
        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=querydict).json()

    commid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    querydict.update([('committee_id',comm_ids[commid])
                     ,('last_index',[])
                     ,('last_contribution_receipt_date',[])])

{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00715532'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00722108'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00713958'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_rece

IndexError: list index out of range

In [75]:
# Concatenate all dfs
itemdf=pd.concat(dfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

In [77]:
#Data cleaning steps
itemdf['contributor_zip'] = itemdf['contributor_zip'].str[:5]

#Filter out 00000, Nulls
itemdf = itemdf[(itemdf['contributor_zip'].isnull()==False)
                 & (itemdf['contributor_zip']!='00000')]

In [78]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/individual-senate-contributions.csv')